# PHYS 3009: TeV Gamma-Ray Data Analysis with GammaPy

This jupyter notebook presents a quick analysis of H.E.S.S. observations of the Crab Nebula. The data set is part of the first public test data release. You can find more information here: https://www.mpi-hd.mpg.de/hfm/HESS/pages/dl3-dr1/

The analysis is performed using gammapy, a community-developed, open-source Python package for gamma-ray astronomy (https://docs.gammapy.org/). More information on the individual data analysis steps can be found in the gammapy tutorials: https://docs.gammapy.org/1.1/tutorials/index.html

# 1. First Look at the Data

## Data download
The following lines check the setup and download the data.

### import
The next cell executes the imports necessary for this project.

In [ ]:
from gammapy.utils.check import check_tutorials_setup

In [ ]:
from gammapy.data import DataStore

In [ ]:
# end import

In [ ]:
check_tutorials_setup()

In [ ]:
data_store = DataStore.from_dir("$GAMMAPY_DATA/1.1/hess-dl3-dr1")

In [ ]:
data_store.info()

Expected output:

```
Data store:
HDU index table:
BASE_DIR: gammapy-data/1.1/hess-dl3-dr1
Rows: 630
OBS_ID: 20136 -- 47829
HDU_TYPE: ['aeff', 'bkg', 'edisp', 'events', 'gti', 'psf']
HDU_CLASS: ['aeff_2d', 'bkg_3d', 'edisp_2d', 'events', 'gti', 'psf_table']


Observation table:
Observatory name: 'N/A'
Number of observations: 105
```

## Run Selection

### imports

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord, Angle

In [ ]:
# end imports

We set the source position. frame='icrs' indicates that we are using coordinates in right ascension and declination.

In [ ]:
source_pos = SkyCoord(83.633*u.deg, 22.014*u.deg, frame='icrs')

In [ ]:
selectradius = 2.5*u.deg

In [ ]:
conesearch = data_store.obs_table.select_sky_circle(source_pos, selectradius)

In [ ]:
conesearch

In [ ]:
runlist = conesearch['OBS_ID'].value

In [ ]:
print(runlist)

In [ ]:
len(runlist)

In [ ]:
observations = data_store.get_observations(runlist)

We create a dictionary where we will store final results which we produce along the way.

In [ ]:
final_results = {}

Let's add some information to our results:

In [ ]:
final_results['run list'] = runlist

## Event List

Let's have a look at the event list of one single run. We will use run number 23523.

In [ ]:
runno = '23523'

### imports

In [ ]:
from gammapy.data import EventList

import matplotlib.pyplot as plt

In [ ]:
# end imports

In [ ]:
events = observations[runno].events

In [ ]:
print(events)

In [ ]:
events.peek()

In [ ]:
events.table

In [ ]:
plt.hist2d(events.table['RA'], 
           events.table['DEC'], 
           #bins = [100,100]
          )

In [ ]:
plt.hist(events.table['ENERGY'],
         #log = True,
         #range = [0.1,100],
         #bins = 100,
        )

## Theta$^2$ Plot

### on-source counts

In [ ]:
events.radec

In [ ]:
first_event = events.radec[0]

In [ ]:
first_event

In [ ]:
first_event.separation(source_pos)

In [ ]:
theta2 = events.radec.separation(source_pos)**2

In [ ]:
print(theta2)

In [ ]:
ret = plt.hist(theta2.value, 
               range = [0,0.1], 
               bins = 50
              )

plt.xlabel('$\\theta^2$ [deg$^2$]')

#plt.savefig('Theta2_on.svg')

In [ ]:
ret

In [ ]:
n = ret[0]
x = ret[1]

In [ ]:
print(x)

In [ ]:
x[1:]<0.01

In [ ]:
n[ x[1:]<0.01 ]

In [ ]:
oncounts = n[x[1:]<0.01].sum()

In [ ]:
oncounts

In [ ]:
final_results['on counts'] = oncounts

### off-source counts

In [ ]:
obs = observations[runno]

In [ ]:
print(obs)

In [ ]:
obs.pointing_radec

In [ ]:
separation = obs.pointing_radec.separation(source_pos)
print (separation)

In [ ]:
position_angle = obs.pointing_radec.position_angle(source_pos)
print (position_angle.to(u.deg))

In [ ]:
offpos = obs.pointing_radec.directional_offset_by( position_angle+180*u.deg, separation)

In [ ]:
offpos

In [ ]:
theta2_off = events.radec.separation(offpos)**2

In [ ]:
plt.hist(theta2.value, range = [0,0.1], bins = 50)
ret_off = plt.hist(theta2_off.value, range = [0,0.1], bins = 50, alpha = 0.5)

plt.xlabel('$\\theta^2$ [deg$^2$]')

#plt.savefig('Theta2_onoff.svg')

In [ ]:
n_off = ret_off[0]
offcounts = n_off[x[1:]<0.01].sum()

In [ ]:
print(oncounts,offcounts)

In [ ]:
final_results['off counts'] = offcounts

### excess

In [ ]:
excess = oncounts - offcounts

In [ ]:
excess

In [ ]:
final_results['excess'] = excess

### background check

Let's check that there is no excess at larger $\theta^2$ values, where we do not expect any emission.

In [ ]:
x[1:] > 0.05

In [ ]:
check_on = n[x[1:]>0.05].sum()

In [ ]:
check_off = n_off[x[1:]>0.05].sum()

In [ ]:
print('{} - {} = {}'.format(check_on, check_off, check_on - check_off))

## Significance

In [ ]:
total = oncounts + offcounts

In [ ]:
total

In [ ]:
#from math import sqrt

from numpy import sqrt

In [ ]:
sigma = sqrt(total)

In [ ]:
sigma

In [ ]:
significance_1 = excess/sigma

In [ ]:
significance_1

In [ ]:
from gammapy.stats import WStatCountsStatistic

In [ ]:
stat = WStatCountsStatistic(n_on=oncounts, n_off=offcounts, alpha=1.)

print('excess: {} \nsignificance: {}'.format(stat.n_sig,stat.sqrt_ts))

Let's check the background:

In [ ]:
stat_check = WStatCountsStatistic(n_on=check_on, n_off=check_off, alpha=1.)

print('excess: {} \nsignificance: {}'.format(stat_check.n_sig,stat_check.sqrt_ts))

## Simple Counts Map

### imports

In [ ]:
from gammapy.maps import Map

In [ ]:
#end imports

In [ ]:
map_crab = Map.create(binsz=0.01*u.deg, 
                      width=(5*u.deg, 5*u.deg), 
                      skydir=source_pos, 
                      frame='icrs')

In [ ]:
map_crab.plot()

In [ ]:
events.radec

In [ ]:
map_crab.fill_by_coord(events.radec)

In [ ]:
map_crab.plot()
#    add_cbar = True,
#    cmap = 'coolwarm'
#)

In [ ]:
smoothed = map_crab.smooth(width=0.05 * u.deg, kernel="gauss")

In [ ]:
smoothed.plot()
#    stretch="log", 
#    add_cbar=True)

In [ ]:
final_results['counts map'] = smoothed

#### Your playground
Make a smoothed sky map. Make the map 3degx3deg large, and use a binning of 0.005deg. Smooth with a disk kernel with 0.03deg. Add a colour bar and stretch such that both the source and the background is clearly seen. You can also try different colour schemes, for example cmap='ocean_r'.

In [ ]:
## your code here

## Your playground

If you want to do it yourself, try the following.

Make a theta^2 plot for signal and background for run no. 23523. Calculate the number of on and off events for a theta^2 cut of 0.02. Calculate the significance.

In [ ]:
## your code here

## Summary

That's all. Let's see what we have.

In [ ]:
final_results

In [ ]:
final_results['counts map'].plot()